In [1]:
import pandas as pd
import jieba
import re

In [2]:
comment_maotai = pd.read_excel('comment_maotai_raw.xlsx',index_col = 0)
comment_maotai.head()

,0,1
0,05-06 08:15,【名博论市】假日多空消息平衡，期望5月开门红
1,05-06 08:59,贵州：拟持续深化领导干部利用茅台酒谋取私利问题专项
2,05-05 16:34,税金确认导致利润波动，全年经营稳健长期价
3,05-06 10:07,茅台又一子公司将谋求IPO 专家：股民反对
4,05-04 23:33,去年白酒为什么涨了几十倍甚至上百倍？刚开


In [3]:
comment_maotai.columns = ['time','comments']
comment_maotai.tail()

,time,comments
479222,07-14 10:01,珠峰山之巅，高处不胜寒
479223,07-14 09:58,几分钟涨停？603879永悦科技
479224,07-14 09:55,600218全柴动力暴涨潜力特大
479225,07-14 09:54,峰有多高，谷有多深，不信你能涨到天上去。
479226,07-14 09:47,今年目标预测250[大笑]


In [4]:
comment_maotai.dtypes

time        object
comments    object
dtype: object

In [5]:
comment_maotai['year'] = None
year = ['2021-','2020-','2019-','2018-','2017-']
n = 0
for i in range(len(comment_maotai.index)):
    mon = int(comment_maotai['time'][i][0:2])
    if mon == 1 and mon != int(comment_maotai['time'][i+1][0:2]):
        comment_maotai['year'][i] = year[n]
        n = n + 1
    else:
        comment_maotai['year'][i] = year[n]

print(round(i/len(comment_maotai.index)*100,2),'% finished')

100.0 % finished


In [6]:
comment_maotai.tail()

,time,comments,year
479222,07-14 10:01,珠峰山之巅，高处不胜寒,2017-
479223,07-14 09:58,几分钟涨停？603879永悦科技,2017-
479224,07-14 09:55,600218全柴动力暴涨潜力特大,2017-
479225,07-14 09:54,峰有多高，谷有多深，不信你能涨到天上去。,2017-
479226,07-14 09:47,今年目标预测250[大笑],2017-


In [7]:
comment_maotai['time']=comment_maotai['time'].apply(str)
comment_maotai['year']=comment_maotai['year'].apply(str)
comment_maotai['comments']=comment_maotai['comments'].apply(str)
comment_maotai['date']=comment_maotai.apply(lambda x:x['year']+x['time'],axis=1)
comment_maotai

,time,comments,year,date
0,05-06 08:15,【名博论市】假日多空消息平衡，期望5月开门红,2021-,2021-05-06 08:15
1,05-06 08:59,贵州：拟持续深化领导干部利用茅台酒谋取私利问题专项,2021-,2021-05-06 08:59
2,05-05 16:34,税金确认导致利润波动，全年经营稳健长期价,2021-,2021-05-05 16:34
3,05-06 10:07,茅台又一子公司将谋求IPO 专家：股民反对,2021-,2021-05-06 10:07
4,05-04 23:33,去年白酒为什么涨了几十倍甚至上百倍？刚开,2021-,2021-05-04 23:33
...,...,...,...,...
479222,07-14 10:01,珠峰山之巅，高处不胜寒,2017-,2017-07-14 10:01
479223,07-14 09:58,几分钟涨停？603879永悦科技,2017-,2017-07-14 09:58
479224,07-14 09:55,600218全柴动力暴涨潜力特大,2017-,2017-07-14 09:55
479225,07-14 09:54,峰有多高，谷有多深，不信你能涨到天上去。,2017-,2017-07-14 09:54


In [8]:
stop_words=pd.read_excel('stop_words.xlsx')['stop_words'].tolist()

In [9]:
comment_maotai['seg']=None
for i in range(len(comment_maotai.index)):
    temp0=comment_maotai['comments'][i]
    temp1=re.sub(r'[a-zA-Z0-9]','',temp0)
    temp2=list(set(jieba.lcut(temp1)).difference(set(stop_words)))
    comment_maotai['seg'][i]=" ".join(str(i) for i in temp2)
    
print(round(i/len(comment_maotai.index)*100,2),'% finished')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/jl/mx0f9lh56s30dz8s3j7l2bk00000gn/T/jieba.cache
Loading model cost 0.419 seconds.
Prefix dict has been built successfully.


100.0 % finished


In [10]:
seg=comment_maotai[['date','seg']]
seg.columns=['created_time','title']
seg.head()

,created_time,title
0,2021-05-06 08:15,消息 期望 平衡 名博论 假日 多空 开门红 市 月
1,2021-05-06 08:59,干部 茅台酒 利用 专项 谋取私利 贵州 领导 深化 持续 拟
2,2021-05-05 16:34,经营 确认 稳健 税金 利润 导致 价 波动 长期
3,2021-05-06 10:07,茅台 子公司 专家 股民 谋求 反对
4,2021-05-04 23:33,去年 几十倍 涨 刚开 百倍 白酒


In [11]:
seg.to_csv('stock_comments_seg2.csv',index=None)

In [12]:
seg.tail()

,created_time,title
479222,2017-07-14 10:01,寒 珠峰 山之巅 高处不胜
479223,2017-07-14 09:58,涨停 科技 几分钟 永悦
479224,2017-07-14 09:55,动力 暴涨 全柴 潜力 特大
479225,2017-07-14 09:54,谷 有多深 峰 涨 高 天上 不信
479226,2017-07-14 09:47,大笑 目标 预测
